In [1]:
import requests
import pandas as pd
from datetime import datetime
import time
from typing import List, Dict
import logging

class PBPStatsAPI:
    def __init__(self, start_year: int = 2013, end_year: int = 2024):
        self.base_url = "https://api.pbpstats.com/get-game-logs/nba?"
        self.season_types = ["Regular Season", "Playoffs"]
        self.start_year = start_year
        self.end_year = end_year

    def get_season_years(self) -> List[str]:
        """Generate season strings from start_year to end_year."""
        return [f"{year-1}-{str(year)[2:]}" for year in range(self.start_year, self.end_year + 1)]
    
    def get_team_game_logs(self, team_id: str) -> pd.DataFrame:
        """
        Fetch all game logs for a specific team within the specified year range.
        
        Args:
            team_id (str): The team ID in PBP Stats format
            
        Returns:
            pd.DataFrame: DataFrame containing all game logs
        """
        all_games = []
        seasons = self.get_season_years()
        
        for season in seasons:
            for season_type in self.season_types:
                if season_type !='Playoffs':
        
                    try:
                        params = {
                            "Season": season,
                            "SeasonType": season_type,
                            "EntityId": team_id,
                            "EntityType": "Team"
                        }
                        
                        response = requests.get(self.base_url, params=params)
                        data_response = response.json()
                        raw_data = data_response["multi_row_table_data"]
                        games_data = pd.DataFrame(raw_data)
                        
                        games_data['Season'] = season
                        games_data['year'] = int(season.split('-')[0]) + 1
                        games_data['SeasonType'] = season_type
                        all_games.append(games_data)
                                
                        # Respect API rate limits
                        time.sleep(3)
                        
                    except requests.exceptions.RequestException as e:
                        logging.error(f"Error fetching data for {team_id} in {season} {season_type}: {str(e)}")
                        continue
                    
        return pd.concat(all_games) if all_games else pd.DataFrame()

def fetch_all_teams_game_logs(team_ids: List[str], start_year: int, end_year: int) -> Dict[str, pd.DataFrame]:
    """
    Fetch game logs for multiple teams within the specified year range.
    
    Args:
        team_ids (List[str]): List of team IDs
        start_year (int): Starting year for data collection
        end_year (int): Ending year for data collection
        
    Returns:
        Dict[str, pd.DataFrame]: Dictionary with team IDs as keys and their game logs as values
    """
    api = PBPStatsAPI(start_year=start_year, end_year=end_year)
    team_games = {}
    
    for team_id in team_ids:
        logging.info(f"Fetching game logs for team {team_id}")
        team_games[team_id] = api.get_team_game_logs(team_id)
        
    return team_games

# Example usage
if __name__ == "__main__":
    # Example team IDs (you'll need to use the correct IDs from PBP Stats)
    
    df = pd.read_csv('index_master.csv')
    df = df[df.year > 2013]
    df = df[df.team != 'TOT']
    team_ids = df['team_id'].unique().tolist()

    # Input start and end years
    start_year = 2025
    end_year =2025

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    
    # Fetch game logs
    team_game_logs = fetch_all_teams_game_logs(team_ids, start_year, end_year)
    frames=[]
    
    # Example: Save to CSV files
    for team_id, games_df in team_game_logs.items():
        if not games_df.empty:
            for year in range(start_year, end_year + 1):
                team_df = games_df[games_df.year == year]
                
                if len(team_df)!=0:
                    team_df.to_csv(f"team/{year}/{team_id}.csv", index=False)
                    frames.append(team_df)
            logging.info(f"Saved game logs for {team_id}")
    
    

2024-12-20 14:21:24,341 - INFO - Fetching game logs for team 1610612761
2024-12-20 14:21:28,280 - INFO - Fetching game logs for team 1610612758
2024-12-20 14:21:32,208 - INFO - Fetching game logs for team 1610612760
2024-12-20 14:21:36,399 - INFO - Fetching game logs for team 1610612766
2024-12-20 14:21:40,339 - INFO - Fetching game logs for team 1610612749
2024-12-20 14:21:44,376 - INFO - Fetching game logs for team 1610612753
2024-12-20 14:21:48,279 - INFO - Fetching game logs for team 1610612740
2024-12-20 14:21:52,151 - INFO - Fetching game logs for team 1610612752
2024-12-20 14:21:56,068 - INFO - Fetching game logs for team 1610612757
2024-12-20 14:22:00,186 - INFO - Fetching game logs for team 1610612755
2024-12-20 14:22:04,134 - INFO - Fetching game logs for team 1610612754
2024-12-20 14:22:08,315 - INFO - Fetching game logs for team 1610612748
2024-12-20 14:22:12,226 - INFO - Fetching game logs for team 1610612763
2024-12-20 14:22:16,167 - INFO - Fetching game logs for team 161

In [26]:
import os
year=2024

for year in range(2015,2026):
    directory = "team/"+str(year)
    files = os.listdir(directory)
    files =[file for file in files if 'game_logs' not in file and '.csv' in file and 'vs' not in file]
    
    totals=[]
    for file in files:
        df=pd.read_csv(directory+'/'+file)
        totals.append(df)
    master= pd.concat(totals)

    team_dict={'OKC': '1610612760',
            'MIL': '1610612749',
            'SAC': '1610612758',
            'LAL': '1610612747',
            'BOS': '1610612738',
            'DEN': '1610612743',
            'MIN': '1610612750',
            'NYK': '1610612752',
            'PHO': '1610612756',
            'PHX': '1610612756',
            'ORL': '1610612753',
            'CHA': '1610612766',
            'CHO': '1610612766',
            'CLE': '1610612739',
            'LAC': '1610612746',
            'ATL': '1610612737',
            'MIA': '1610612748',
            'DAL': '1610612742',
            'DET': '1610612765',
            'MEM': '1610612763',
            'TOR': '1610612761',
            'CHI': '1610612741',
            'IND': '1610612754',
            'SAS': '1610612759',
            'HOU': '1610612745',
            'BRK': '1610612751',
            'BKN': '1610612751',
            'WAS': '1610612764',
            'GSW': '1610612744',
            'PHI': '1610612755',
            'UTA': '1610612762',
            'POR': '1610612757',
            'NOP': '1610612740'}
    master.dropna(subset='Opponent',inplace=True)
    for opp in master.Opponent.unique():
        teamframe=master[master.Opponent==opp].reset_index()

 
        teamframe.to_csv(directory+'/'+team_id+'vs.csv',index=False)
        
    
    print(year)

   Opponent
0       LAL
1       LAL
2       LAL
3       LAL
4       LAL
..      ...
77      LAL
78      LAL
79      LAL
80      LAL
81      LAL

[82 rows x 1 columns]
   Opponent
0       SAS
1       SAS
2       SAS
3       SAS
4       SAS
..      ...
84      SAS
85      SAS
86      SAS
87      SAS
88      SAS

[89 rows x 1 columns]
   Opponent
0       UTA
1       UTA
2       UTA
3       UTA
4       UTA
..      ...
77      UTA
78      UTA
79      UTA
80      UTA
81      UTA

[82 rows x 1 columns]
   Opponent
0       MEM
1       MEM
2       MEM
3       MEM
4       MEM
..      ...
88      MEM
89      MEM
90      MEM
91      MEM
92      MEM

[93 rows x 1 columns]
   Opponent
0       SAC
1       SAC
2       SAC
3       SAC
4       SAC
..      ...
77      SAC
78      SAC
79      SAC
80      SAC
81      SAC

[82 rows x 1 columns]
    Opponent
0        GSW
1        GSW
2        GSW
3        GSW
4        GSW
..       ...
98       GSW
99       GSW
100      GSW
101      GSW
102      GSW

[103 row